In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import imageio
import cv2

paths = pathlib.Path('./Optical Micrographs').glob('*H.jpg')
paths_sorted = [x for x in paths]
paths_sorted.sort()

def toBW(img):
    (thresh, blackAndWhiteImage) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    return blackAndWhiteImage

originals = [cv2.imread(str(path)) for path in paths_sorted]
bws = [toBW(original) for original in originals]
#arr = np.array([np.array(i, dtype=bool) for i in im])

#Question for Zhouer: any suggestion on how to measure roundness?

/var/folders/q8/tcfvx04d1y11jv7mj6t996fr0000gn/T/ipykernel_44445/4144059007.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([np.array(i, dtype=bool) for i in im])


In [ ]:
def getCountours(img):
    contours, hierarchy = cv2.findContours(~img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def convertToDataArray(contour, index):
    area = cv2.contourArea(contour)
    perimiter = cv2.arcLength(contour, True)
    (cx, cy), (w, h), c = cv2.minAreaRect(contour)
    print([index, area, perimiter, w, h, cx, cy])

def paintContours(img, cnts):
    for cnt in contours:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(img,[box],0,(0,0,255),5)

contours = [getCountours(i) for i in bws]

paintContours(originals[0], contours[0])
cv2.imwrite("Fixed.jpg", originals[0])


#We want source image, area, length, height, location
#data = [[convertToDataArray(cnt, index) for cnt in item] for index, item in enumerate(contours)]

In [61]:
for cnt in contours[0]:
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(im[0],[box],0,(0,255,255),2)


True